In [19]:
import psycopg2
import pandas as pd

conn = psycopg2.connect(host="35.188.222.122", port = "5432", database="postgres", user="epic_einstein", password="#pbscbg13!")

cur = conn.cursor()

cur.execute("""SELECT * FROM security_reference""")
query_results = cur.fetchall()
print(query_results)

cur.execute("""SELECT * FROM daily_returns""")
query_results = cur.fetchall()
daily_returns_df = pd.DataFrame(query_results) 
daily_returns_df.to_csv('daily_returns.csv')

cur.close()
conn.close()

[('Apple Inc.', 'AAPL-US', 'US0378331005'), ('Microsoft Corporation', 'MSFT-US', 'US5949181045'), ('Intel Corporation', 'INTC-US', 'US4581401001'), ('Mastercard Incorporated Class A', 'MA-US', 'US57636Q1040'), ('JPMorgan Chase & Co.', 'JPM-US', 'US46625H1005'), ('Visa Inc. Class A', 'V-US', 'US92826C8394'), ('Procter & Gamble Company', 'PG-US', 'US7427181091'), ('UnitedHealth Group Incorporated', 'UNH-US', 'US91324P1021'), ('Verizon Communications Inc.', 'VZ-US', 'US92343V1044'), ('Johnson & Johnson', 'JNJ-US', 'US4781601046'), ('Alphabet Inc. Class A', 'GOOGL-US', 'US02079K3059'), ('Alphabet Inc. Class C', 'GOOG-US', 'US02079K1079'), ('AT&T Inc.', 'T-US', 'US00206R1023'), ('Home Depot, Inc.', 'HD-US', 'US4370761029'), ('Facebook, Inc. Class A', 'FB-US', 'US30303M1027'), ('Amazon.com, Inc.', 'AMZN-US', 'US0231351067'), ('Alphabet Inc. Class C', 'GOOG-US', 'US38259P7069'), ('PayPal Holdings Inc', 'PYPL-US', 'US70450Y1038'), ('Adobe Inc.', 'ADBE-US', 'US00724F1012'), ('Alphabet Inc. Clas

In [202]:
import psycopg2
import pandas as pd

conn = psycopg2.connect(host="35.188.222.122", port = "5432", database="postgres", user="epic_einstein", password="#pbscbg13!")

cur = conn.cursor()

def create_pandas_table(sql_query, database = conn):
    table = pd.read_sql_query(sql_query, database)
    return table

daily_returns = create_pandas_table("SELECT * FROM security_reference INNER JOIN daily_returns USING (isin_code)")
daily_returns_df = pd.DataFrame(daily_returns) 
daily_returns_df['year'] = pd.DatetimeIndex(daily_returns_df['p_date']).year


Compound_Total_Return = daily_returns_df.groupby(['proper_name'])[['one_day_pct']].apply(lambda x: ((x+1).cumprod()**(1/len(x))-1).iloc[-1])


print(Compound_Total_Return)

cur.close()
conn.close()

                                 one_day_pct
proper_name                                 
AT&T Inc.                                NaN
Adobe Inc.                          0.157132
Alphabet Inc. Class A                    NaN
Alphabet Inc. Class C                    NaN
Amazon.com, Inc.                         NaN
Apple Inc.                          0.071880
Berkshire Hathaway Inc. Class B    -0.089744
Facebook, Inc. Class A                   NaN
Home Depot, Inc.                         NaN
Intel Corporation                        NaN
JPMorgan Chase & Co.                     NaN
Johnson & Johnson                        NaN
Mastercard Incorporated Class A     0.086500
Microsoft Corporation                    NaN
NVIDIA Corporation                       NaN
PayPal Holdings Inc                 0.137836
Procter & Gamble Company                 NaN
UnitedHealth Group Incorporated    -0.007392
Verizon Communications Inc.              NaN
Visa Inc. Class A                        NaN


In [248]:
import psycopg2
import pandas as pd

conn = psycopg2.connect(host="35.188.222.122", port = "5432", database="postgres", user="epic_einstein", password="#pbscbg13!")

cur = conn.cursor()

def create_pandas_table(sql_query, database = conn):
    table = pd.read_sql_query(sql_query, database)
    return table

daily_returns = create_pandas_table("SELECT * FROM security_reference INNER JOIN daily_returns USING (isin_code)")
daily_returns_df = pd.DataFrame(daily_returns) 
daily_returns_df['year'] = pd.DatetimeIndex(daily_returns_df['p_date']).year


#1)  Compound total return (Formula from here:  https://www.youtube.com/watch?v=yFABQonqMfU)

daily_returns_df['1+one_day_pct'] = (daily_returns_df['one_day_pct']/100) + 1
daily_returns_df['CompTotalReturn'] = (daily_returns_df.groupby(['proper_name'])[['1+one_day_pct']].cumprod()) - 1

Comp_Total_return = daily_returns_df.groupby('proper_name')['proper_name','CompTotalReturn'].tail(1)

print(Comp_Total_return)

#2)  Compound annual growth rate 

daily_returns_df['CAGR'] = (daily_returns_df.groupby(['proper_name', 'year'])[['1+one_day_pct']].cumprod()) - 1
Comp_annual_growth_rate = daily_returns_df.groupby(['proper_name', 'year'])['proper_name','year','CAGR'].tail(1)

print(Comp_annual_growth_rate)

#3)  Alpha 

daily_returns_df.to_csv('daily_returns.csv')

cur.close()
conn.close()

                           proper_name  CompTotalReturn
1261                        Adobe Inc.         3.023772
2523             Alphabet Inc. Class A         1.130762
3785             Alphabet Inc. Class C         1.185282
5047                  Amazon.com, Inc.         3.542371
6309                        Apple Inc.         1.112504
7571                         AT&T Inc.         0.093482
8833   Berkshire Hathaway Inc. Class B         0.278531
10095           Facebook, Inc. Class A         1.106327
11357                 Home Depot, Inc.         0.879222
12619                Intel Corporation         0.788170
13881                Johnson & Johnson         0.490253
15143             JPMorgan Chase & Co.         0.550879
16405  Mastercard Incorporated Class A         1.701065
17667            Microsoft Corporation         2.708159
18929               NVIDIA Corporation        11.256943
20165              PayPal Holdings Inc         1.608004
21427         Procter & Gamble Company         0

C:\Users\singh\.conda\envs\tensorflow\lib\site-packages\ipykernel_launcher.py:22: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
C:\Users\singh\.conda\envs\tensorflow\lib\site-packages\ipykernel_launcher.py:29: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.


In [315]:
import psycopg2
import pandas as pd
import pandas_datareader as web
from scipy import stats

conn = psycopg2.connect(host="35.188.222.122", port = "5432", database="postgres", user="epic_einstein", password="#pbscbg13!")

cur = conn.cursor()

def create_pandas_table(sql_query, database = conn):
    table = pd.read_sql_query(sql_query, database)
    return table

daily_returns = create_pandas_table("SELECT * FROM security_reference INNER JOIN daily_returns USING (isin_code)")
daily_returns_df = pd.DataFrame(daily_returns) 
daily_returns_df['year'] = pd.DatetimeIndex(daily_returns_df['p_date']).year


#1)  Compound total return (Formula from here:  https://www.youtube.com/watch?v=yFABQonqMfU)

daily_returns_df['1+one_day_pct'] = (daily_returns_df['one_day_pct']/100) + 1
daily_returns_df['CompTotalReturn'] = (daily_returns_df.groupby(['proper_name'])[['1+one_day_pct']].cumprod()) - 1

Comp_Total_return = daily_returns_df.groupby('proper_name')['proper_name','CompTotalReturn'].tail(1)

#print(Comp_Total_return)

#2)  Compound annual growth rate 

daily_returns_df['CAGR'] = (daily_returns_df.groupby(['proper_name', 'year'])[['1+one_day_pct']].cumprod()) - 1
Comp_annual_growth_rate = daily_returns_df.groupby(['proper_name', 'year'])['proper_name','year','CAGR'].tail(1)

#print(Comp_annual_growth_rate)

#3)  Alpha 

SPY_price = web.get_data_yahoo('SPY',start = '2015-03-26',end = '2020-03-31')                             
SPY_ret = SPY_price["Adj Close"].pct_change()[1:]

Ticker_list = daily_returns_df['proper_name'].unique().tolist()
Ticker_list.remove('PayPal Holdings Inc')
daily_returns_df['one_day_pct_adj'] = (daily_returns_df['one_day_pct']/100)

row = []
print ("Ticker" + "            Beta         " + "     Alpha")

for i in Ticker_list:
    ticker_pct_change = daily_returns_df[daily_returns_df['proper_name'] == i]
    ticker_pct_change = ticker_pct_change['one_day_pct_adj']
    (beta, alpha) = stats.linregress(SPY_ret.values, ticker_pct_change.values)[0:2]
    row = i + "  " + str(beta) + str(alpha)
    print (row)

daily_returns_df.to_csv('daily_returns.csv')

cur.close()
conn.close()

C:\Users\singh\.conda\envs\tensorflow\lib\site-packages\ipykernel_launcher.py:24: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
C:\Users\singh\.conda\envs\tensorflow\lib\site-packages\ipykernel_launcher.py:31: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.


Ticker            Beta              Alpha
Adobe Inc.  0.51570518631778420.0011152633848867592
Alphabet Inc. Class A  0.40425979359216690.0006017104565940862
Alphabet Inc. Class C  0.399383533260054770.000625076422779753
Amazon.com, Inc.  0.275917093280556360.0012768875196431512
Apple Inc.  0.461022909831185770.0005985934166516665
AT&T Inc.  0.374532549275313653.960298637646517e-05
Berkshire Hathaway Inc. Class B  0.430909167967112870.00013719367159853883
Facebook, Inc. Class A  0.419849439381782430.0006418825583153348
Home Depot, Inc.  0.50979292274985840.0004543983101330503
Intel Corporation  0.54405413656107930.0004720582646155147
Johnson & Johnson  0.32040624811860090.00028602565567250547
JPMorgan Chase & Co.  0.58881710536458380.00029965775250839397
Mastercard Incorporated Class A  0.50254863164983130.0007615498409620037
Microsoft Corporation  0.51703621374577310.0010111120073381314
NVIDIA Corporation  0.63587888141991270.0021938354990180863
Procter & Gamble Company  0.387238374850

In [334]:
import psycopg2
import pandas as pd
import pandas_datareader as web
from scipy import stats
import statistics

conn = psycopg2.connect(host="35.188.222.122", port = "5432", database="postgres", user="epic_einstein", password="#pbscbg13!")

cur = conn.cursor()

def create_pandas_table(sql_query, database = conn):
    table = pd.read_sql_query(sql_query, database)
    return table

daily_returns = create_pandas_table("SELECT * FROM security_reference INNER JOIN daily_returns USING (isin_code)")
daily_returns_df = pd.DataFrame(daily_returns) 
daily_returns_df['year'] = pd.DatetimeIndex(daily_returns_df['p_date']).year


#1)  Compound total return (Formula from here:  https://www.youtube.com/watch?v=yFABQonqMfU)

daily_returns_df['1+one_day_pct'] = (daily_returns_df['one_day_pct']/100) + 1
daily_returns_df['CompTotalReturn'] = (daily_returns_df.groupby(['proper_name'])[['1+one_day_pct']].cumprod()) - 1

Comp_Total_return = daily_returns_df.groupby('proper_name')['proper_name','CompTotalReturn'].tail(1)

#print(Comp_Total_return)

#2)  Compound annual growth rate 

daily_returns_df['CAGR'] = (daily_returns_df.groupby(['proper_name', 'year'])[['1+one_day_pct']].cumprod()) - 1
Comp_annual_growth_rate = daily_returns_df.groupby(['proper_name', 'year'])['proper_name','year','CAGR'].tail(1)

#print(Comp_annual_growth_rate)

#3)  Alpha, Beta (Based on SPY benchmark, data pulled from Yahoo, used line regression to calculate metrics) 

SPY_price = web.get_data_yahoo('SPY',start = '2015-03-26',end = '2020-03-31')                             
SPY_ret = SPY_price["Adj Close"].pct_change()[1:]

Ticker_list = daily_returns_df['proper_name'].unique().tolist()
Ticker_list.remove('PayPal Holdings Inc')
daily_returns_df['one_day_pct_adj'] = (daily_returns_df['one_day_pct']/100)

row = []
#print ("Ticker" + "            Beta         " + "     Alpha")

for i in Ticker_list:
    ticker_pct_change = daily_returns_df[daily_returns_df['proper_name'] == i]
    ticker_pct_change = ticker_pct_change['one_day_pct_adj']
    (beta, alpha) = stats.linregress(SPY_ret.values, ticker_pct_change.values)[0:2]
    row = i + "  " + str(beta) + str(alpha)
    Ticker_average_return = statistics.mean(ticker_pct_change)
    SPY_average_return = statistics.mean(SPY_ret)
    Ticker_std = np.std(ticker_pct_change)
    Sharpe_Ratio = (Ticker_average_return - SPY_average_return) / Ticker_std  # Sharpe ratio formula :  https://www.educba.com/sharpe-ratio-formula/
    print (i + "        Sharpe Ratio        " + "%.6f" %Sharpe_Ratio)


daily_returns_df.to_csv('daily_returns.csv')

cur.close()
conn.close()

C:\Users\singh\.conda\envs\tensorflow\lib\site-packages\ipykernel_launcher.py:25: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
C:\Users\singh\.conda\envs\tensorflow\lib\site-packages\ipykernel_launcher.py:32: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.


Adobe Inc.        Sharpe Ratio        0.050762
Alphabet Inc. Class A        Sharpe Ratio        0.025072
Alphabet Inc. Class C        Sharpe Ratio        0.026217
Amazon.com, Inc.        Sharpe Ratio        0.057088
Apple Inc.        Sharpe Ratio        0.024171
AT&T Inc.        Sharpe Ratio        -0.012167
Berkshire Hathaway Inc. Class B        Sharpe Ratio        -0.003655
Facebook, Inc. Class A        Sharpe Ratio        0.023620
Home Depot, Inc.        Sharpe Ratio        0.019372
Intel Corporation        Sharpe Ratio        0.016790
Johnson & Johnson        Sharpe Ratio        0.005497
JPMorgan Chase & Co.        Sharpe Ratio        0.009909
Mastercard Incorporated Class A        Sharpe Ratio        0.036376
Microsoft Corporation        Sharpe Ratio        0.051243
NVIDIA Corporation        Sharpe Ratio        0.072003
Procter & Gamble Company        Sharpe Ratio        0.011301
UnitedHealth Group Incorporated        Sharpe Ratio        0.027250
Verizon Communications Inc.       